In [2]:
# Cell 1: Installations and Imports
# =================================
!pip install opendatasets kaggle pandas tensorflow scikit-learn opencv-python -q

import os
import numpy as np
import cv2
import random
import gc
import pickle
import json
import opendatasets as od
import pandas as pd # Added pandas for timestamp in consolidation
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='keras.*')

print("✅ All imports successful!")

# Cell 2: Configuration and Dataset Selection
# ===========================================
def get_dataset_choice():
    print("Available datasets:")
    print("1. plants")
    print("2. aid")
    print("3. lc25000")

    while True:
        choice = input("Select dataset (enter number or name): ").strip().lower()

        # Handle numeric input
        if choice == "1":
            return "plants"
        elif choice == "2":
            return "aid"
        elif choice == "3":
            return "lc25000"
        elif choice in ["plants", "aid", "lc25000"]:
            return choice
        else:
            print("Invalid choice. Please enter 1, 2, 3, or the dataset name.")

DATASET_CHOICE = get_dataset_choice()
print(f"Selected dataset: {DATASET_CHOICE}")

# Ask user about their environment
def get_environment():
    while True:
        env = input("Are you running this code in Google Colab or locally? (colab/local): ").lower().strip()
        if env in ['colab', 'local']:
            return env
        else:
            print("Please enter either 'colab' or 'local'")

environment = get_environment()

# Dataset-specific configurations
DATASET_CONFIGS = {
    "plants": {
        "dataset_url": "https://www.kaggle.com/datasets/yudhaislamisulistya/plants-type-datasets",
        "download_folder": "./plant-datasets",
        "train_path": "",
        "val_path": "",
        "test_path": "",
        "features_dir": "./saved_features_plants",
        "split_type": "pre_split"
    },
    "aid": {
        "dataset_url": "https://www.kaggle.com/datasets/jiayuanchengala/aid-scene-classification-datasets",
        "download_folder": "./aid-datasets",
        "data_root": "",
        "features_dir": "./saved_features_aid",
        "split_type": "70_10_20"
    },
    "lc25000": {
        "dataset_url": "https://www.kaggle.com/datasets/javaidahmadwani/lc25000",
        "download_folder": "./lc25000-datasets",
        "data_root": "",
        "features_dir": "./saved_features_lc25000",
        "split_type": "lc25000_split"
    }
}

# Set paths based on environment
if environment == "colab":
    # Colab paths
    DATASET_CONFIGS["plants"]["train_path"] = "/content/plant-datasets/plants-type-datasets/split_ttv_dataset_type_of_plants/Train_Set_Folder"
    DATASET_CONFIGS["plants"]["val_path"] = "/content/plant-datasets/plants-type-datasets/split_ttv_dataset_type_of_plants/Validation_Set_Folder"
    DATASET_CONFIGS["plants"]["test_path"] = "/content/plant-datasets/plants-type-datasets/split_ttv_dataset_type_of_plants/Test_Set_Folder"
    DATASET_CONFIGS["aid"]["data_root"] = "./aid-datasets/aid-scene-classification-datasets/AID"
    DATASET_CONFIGS["lc25000"]["data_root"] = "./lc25000-datasets/lc25000/lung_colon_image_set"

else:  # local environment
    print("\nFor local execution, please ensure you have the following folders in your current directory:")
    print("1. split_ttv_dataset_type_of_plants")
    print("2. AID")
    print("3. lung_colon_image_set")

    # Local paths
    DATASET_CONFIGS["plants"]["train_path"] = "./split_ttv_dataset_type_of_plants/Train_Set_Folder"
    DATASET_CONFIGS["plants"]["val_path"] = "./split_ttv_dataset_type_of_plants/Validation_Set_Folder"
    DATASET_CONFIGS["plants"]["test_path"] = "./split_ttv_dataset_type_of_plants/Test_Set_Folder"
    DATASET_CONFIGS["aid"]["data_root"] = "./AID"
    DATASET_CONFIGS["lc25000"]["data_root"] = "./lung_colon_image_set"


# Get current config
CONFIG = DATASET_CONFIGS[DATASET_CHOICE]

# Common parameters
IMG_SIZE = (128, 128)
VGG_TARGET_SIZE = (224, 224)
PCA_VARIANCE = 0.95

# GA Parameters (kept for reference, but GWO will use its own)
GA_POPULATION_SIZE = 10
GA_MAX_GENERATION = 40
GA_CROSSOVER_RATE = 0.8
GA_MUTATION_RATE = 0.2

# Training Parameters
FEATURE_EXTRACTION_BATCH_SIZE = 128
BATCH_SIZE = 64
EVAL_EPOCHS = 10
FINAL_EPOCHS = 100 # Increased for better convergence
RANDOM_SEED = 42

# Set seeds
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Create features directory
os.makedirs(CONFIG['features_dir'], exist_ok=True)

print(f"✅ Configuration set for {DATASET_CHOICE.upper()} dataset!")

# Cell 3: Dataset Download
# ========================
def download_dataset():
    """Download dataset based on configuration"""
    # ⚠️ REPLACE WITH YOUR ACTUAL KAGGLE CREDENTIALS ⚠️
    kaggle_credentials = {"username": 'yuvanrajvengaladas', "key": '6fd3ece8111002ccca9494a6d7e6212e'}

    with open("kaggle.json", "w") as f:
        json.dump(kaggle_credentials, f)

    try:
        with open("kaggle.json", 'r') as f:
            credentials = json.load(f)
        os.environ['KAGGLE_USERNAME'] = credentials['username']
        os.environ['KAGGLE_KEY'] = credentials['key']

        print(f"Downloading {DATASET_CHOICE} dataset: {CONFIG['dataset_url']}")
        od.download(CONFIG['dataset_url'], data_dir=CONFIG['download_folder'])
        print(f"✅ {DATASET_CHOICE.upper()} dataset downloaded successfully!")

    except Exception as e:
        print(f"❌ Error downloading dataset: {e}")
        print("👉 Please ensure your kaggle.json file has correct credentials")


# Download the dataset
if environment == "colab":
    download_dataset()

# Cell 4: Data Loading and Splitting Functions
# ============================================
def load_images_from_folder(folder, img_size=IMG_SIZE, max_samples=None):
    """Load images from folder structure (for pre-split datasets)"""
    X, y, mapping = [], [], {}
    class_folders = sorted([f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))])

    for idx, cname in enumerate(class_folders):
        mapping[idx] = cname
        files = glob(os.path.join(folder, cname, "*.jpg")) + glob(os.path.join(folder, cname, "*.png")) + glob(os.path.join(folder, cname, "*.jpeg"))

        if max_samples:
            files = files[:max_samples]

        for f in files:
            img = cv2.imread(f)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            y.append(idx)

    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    print(f"✅ Loaded {len(X)} images from {folder}")
    return X, y, mapping

def get_all_filepaths_and_labels(data_root):
    """Gather all image file paths and labels (for non-split datasets)"""
    all_files, all_labels, mapping = [], [], {}
    class_folders = sorted([f for f in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, f))])

    for idx, cname in enumerate(class_folders):
        mapping[idx] = cname
        class_dir = os.path.join(data_root, cname)
        files = glob(os.path.join(class_dir, "*.jpg")) + glob(os.path.join(class_dir, "*.png"))
        all_files.extend(files)
        all_labels.extend([idx] * len(files))

    print(f"📁 Found {len(all_files)} images across {len(mapping)} classes.")
    return np.array(all_files), np.array(all_labels), mapping

def load_images_from_paths(paths, labels, img_size=IMG_SIZE, max_samples=None):
    """Load images from file paths"""
    X, valid_labels = [], []
    if max_samples:
        indices = np.random.choice(len(paths), min(max_samples, len(paths)), replace=False)
        paths = paths[indices]
        labels = labels[indices]

    for i, f in enumerate(paths):
        img = cv2.imread(f)
        if img is None:
            continue
        img = cv2.resize(img, img_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        X.append(img)
        valid_labels.append(labels[i])

    X = np.array(X)
    y = np.array(valid_labels).reshape(-1, 1)
    print(f"✅ Loaded {len(X)} images from paths.")
    return X, y

def load_dataset_plants():
    """Load Plants dataset (pre-split)"""
    # Reduced sample counts for efficiency/testing purposes
    X_train, y_train, mapping = load_images_from_folder(CONFIG['train_path'], max_samples=1000)
    X_val, y_val, _ = load_images_from_folder(CONFIG['val_path'], max_samples=300)
    X_test, y_test, _ = load_images_from_folder(CONFIG['test_path'], max_samples=300)
    return X_train, y_train, X_val, y_val, X_test, y_test, mapping

def load_dataset_aid():
    """Load AID dataset and split 70-10-20"""
    X_paths, y_labels, mapping = get_all_filepaths_and_labels(CONFIG['data_root'])

    # Split: 70% Train, 15% Validation, 15% Test
    X_train_paths, X_temp_paths, y_train_labels, y_temp_labels = train_test_split(
        X_paths, y_labels, test_size=0.3, random_state=RANDOM_SEED, stratify=y_labels
    )
    X_val_paths, X_test_paths, y_val_labels, y_test_labels = train_test_split(
        X_temp_paths, y_temp_labels, test_size=0.5, random_state=RANDOM_SEED, stratify=y_temp_labels
    )

    # Load images (with reduced max_samples for efficiency)
    X_train, y_train = load_images_from_paths(X_train_paths, y_train_labels, max_samples=2000)
    X_val, y_val = load_images_from_paths(X_val_paths, y_val_labels, max_samples=500)
    X_test, y_test = load_images_from_paths(X_test_paths, y_test_labels, max_samples=500)

    return X_train, y_train, X_val, y_val, X_test, y_test, mapping

def load_dataset_lc25000():
    """Load LC25000 dataset with custom split"""
    test_folder = os.path.join(CONFIG['data_root'], "Test Set")
    train_val_folder = os.path.join(CONFIG['data_root'], "Train and Validation Set")

    # Load test set (reduced max_samples for efficiency)
    X_test, y_test, mapping = load_images_from_folder(test_folder, max_samples=500)

    # Load train-val set and split 80-20 (reduced max_samples for efficiency)
    X_train_val, y_train_val, mapping = load_images_from_folder(train_val_folder, max_samples=2500)

    # Split train-val into 80% train, 20% validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.2, random_state=RANDOM_SEED, stratify=y_train_val
    )

    return X_train, y_train, X_val, y_val, X_test, y_test, mapping

# Main data loading function
def load_datasets():
    """Main function to load datasets based on choice"""
    print(f"📂 Loading {DATASET_CHOICE.upper()} dataset...")

    if CONFIG['split_type'] == "pre_split":
        return load_dataset_plants()
    elif CONFIG['split_type'] == "70_10_20":
        return load_dataset_aid()
    elif CONFIG['split_type'] == "lc25000_split":
        return load_dataset_lc25000()
    else:
        raise ValueError(f"Unknown split type: {CONFIG['split_type']}")

print("✅ Data loading functions defined!")

# Cell 5: Feature Extraction Utilities
# ====================================
def preprocess_single_image_optimized(img):
    """Enhanced preprocessing for VGG16 with simple augmentation (horizontal flip)"""
    resized_img = cv2.resize(img, VGG_TARGET_SIZE)
    # Simple data augmentation for better feature generalization
    if random.random() > 0.5:
        resized_img = cv2.flip(resized_img, 1)
    processed_img = preprocess_input(resized_img.astype('float32'))
    return processed_img

def extract_features_optimized(X, dataset_name, batch_size=32):
    """Optimized feature extraction using VGG16"""
    feature_file = os.path.join(CONFIG['features_dir'], f"{dataset_name}_features.pkl")

    if os.path.exists(feature_file):
        print(f"📁 Loading cached features for {dataset_name}...")
        with open(feature_file, 'rb') as f:
            return pickle.load(f)

    print(f"🔧 Extracting features for {dataset_name}...")
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    feature_extractor = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D()
    ])

    features = []
    total_batches = (len(X) + batch_size - 1) // batch_size

    for i in range(0, len(X), batch_size):
        batch_images = []
        for img in X[i:i+batch_size]:
            processed_img = preprocess_single_image_optimized(img)
            batch_images.append(processed_img)

        batch = np.array(batch_images)
        batch_features = feature_extractor.predict(batch, verbose=0)
        features.append(batch_features)

        # Progress tracking
        batch_num = i // batch_size + 1
        if batch_num % 10 == 0 or batch_num == total_batches:
            print(f"    Processed {batch_num}/{total_batches} batches")

        del batch, batch_images
        gc.collect()

    features = np.vstack(features)

    # Save features
    with open(feature_file, 'wb') as f:
        pickle.dump(features, f)

    # Cleanup
    del feature_extractor, base_model
    tf.keras.backend.clear_session()
    gc.collect()

    print(f"💾 Saved features for {dataset_name}: {features.shape}")
    return features

def load_and_extract_features():
    """Complete feature extraction pipeline"""
    print("🚀 Starting feature extraction pipeline...")

    # Load datasets
    X_train_img, y_train, X_val_img, y_val, X_test_img, y_test, mapping = load_datasets()

    print(f"🎯 Classes found: {len(mapping)}")
    print(f"📊 Dataset sizes - Train: {X_train_img.shape[0]}, Val: {X_val_img.shape[0]}, Test: {X_test_img.shape[0]}")

    # Extract features
    X_train_feats = extract_features_optimized(X_train_img, "train", batch_size=FEATURE_EXTRACTION_BATCH_SIZE)
    X_val_feats = extract_features_optimized(X_val_img, "val", batch_size=FEATURE_EXTRACTION_BATCH_SIZE)
    X_test_feats = extract_features_optimized(X_test_img, "test", batch_size=FEATURE_EXTRACTION_BATCH_SIZE)

    # Clean up image data
    del X_train_img, X_val_img, X_test_img
    gc.collect()

    print("✅ Feature extraction completed!")
    return X_train_feats, y_train, X_val_feats, y_val, X_test_feats, y_test, mapping

print("✅ Feature extraction utilities defined!")

# Cell 6: Structured Sparsity Norm (SSN) Implementation
# ====================================================
class StructuredSparsityNorm(tf.keras.regularizers.Regularizer):
    def __init__(self, lambda1=1e-4, lambda2=1e-4):
        self.lambda1 = lambda1
        self.lambda2 = lambda2

    def __call__(self, x):
        # ||W||_1,1-norm: sum of all absolute weights
        l11_norm = self.lambda1 * tf.reduce_sum(tf.abs(x))

        # ||W||_2,1-norm: sum of the Euclidean norm of each row (column group)
        # Using axis=1 assumes features (inputs) are columns, and output units are rows.
        # This implementation assumes weight matrix W has shape (input_dim, hidden_units)
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(x), axis=0) + 1e-8) # L2 norm across input dimension
        l21_norm = self.lambda2 * tf.reduce_sum(l2_norm)

        return l11_norm + l21_norm

    def get_config(self):
        return {"lambda1": self.lambda1, "lambda2": self.lambda2}

def build_ssn_perceptron(input_dim, n_classes, hidden_units=512, dropout_rate=0.5, lr=1e-3):
    """Build ANN with Structured Sparsity Norm regularization"""
    tf.keras.backend.clear_session()
    ssn_reg = StructuredSparsityNorm(lambda1=1e-4, lambda2=1e-4)

    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(hidden_units, activation='relu', kernel_regularizer=ssn_reg))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_units//2, activation='relu', kernel_regularizer=ssn_reg))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_classes, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def evaluate_with_ssn(model, X_val, y_val):
    """Evaluate feature subset using Structured Sparsity Norm concept"""
    preds_proba = model.predict(X_val, verbose=0)
    preds = np.argmax(preds_proba, axis=1)
    classification_error = 1 - accuracy_score(y_val.flatten(), preds)

    # Get model weights for SSN evaluation
    weights = []
    for layer in model.layers:
        if hasattr(layer, 'kernel') and layer.kernel is not None:
            weights.append(layer.kernel.numpy())

    if len(weights) > 0:
        # We only consider the SSN on the first layer's weights for feature selection context
        weight_matrix = weights[0]
        l11_penalty = np.sum(np.abs(weight_matrix))
        # Note: Axis=0 here corresponds to L2 norm of the groups (rows in the SSN table image, columns in Keras weights)
        l21_penalty = np.sum(np.sqrt(np.sum(weight_matrix**2, axis=0)))

        # SSN Evaluation = Classification Error + Regularization Penalty
        ssn_evaluation = classification_error + 1e-4 * (l11_penalty + l21_penalty)
    else:
        ssn_evaluation = classification_error

    return ssn_evaluation, classification_error

print("✅ SSN implementation complete!")

# Cell 7: GWO-ANN-SSN Feature Selector Class (Replacing GA)
# =========================================================

class GWO_ANN_SSN_FeatureSelector:
    def __init__(self, population_size=20, max_generations=100):
        self.population_size = population_size
        self.max_generations = max_generations
        self.best_alpha_pos = None

    def initialize_population(self, n_features):
        """Initializes the population (gray wolves) randomly in continuous space [0, 1]."""
        population = np.random.rand(self.population_size, n_features)
        return population

    def select_feature_subset(self, chromosome, X_data):
        """Converts the continuous wolf position to a binary feature mask using a threshold."""
        # Binarization using a simple threshold
        binary_mask = (chromosome > 0.5).astype(int)
        selected_indices = np.where(binary_mask == 1)[0]

        if len(selected_indices) == 0:
            # Fallback: select a random single feature if none are selected
            selected_indices = [np.random.randint(0, X_data.shape[1])]
            binary_mask = np.zeros(X_data.shape[1], dtype=int)
            binary_mask[selected_indices[0]] = 1

        return X_data[:, selected_indices], selected_indices, binary_mask

    def classify_with_perceptron(self, X_train, y_train, X_val, y_val, n_classes, selected_indices):
        """Wrapper for training the ANN-SSN model on the feature subset."""
        if len(selected_indices) == 0:
            return None, 1.0

        try:
            model = build_ssn_perceptron(len(selected_indices), n_classes, hidden_units=256)
            early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)

            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=EVAL_EPOCHS,
                batch_size=32,
                verbose=0,
                callbacks=[early_stop]
            )
            return model, history
        except Exception as e:
            # print(f"⚠️ Model training failed: {e}")
            return None, 1.0

    def evaluate_with_ssn_norm(self, model, X_val, y_val, feature_ratio):
        """Calculates the fitness value using classification error and feature ratio."""
        if model is None:
            return 1.0 # Max fitness (worst score)

        try:
            ssn_score, classification_error = evaluate_with_ssn(model, X_val, y_val)

            # Fitness = w1 * Error + w2 * Feature_Ratio
            if DATASET_CHOICE == "aid":
                 fitness = 0.9 * classification_error + 0.1 * feature_ratio
            else:
                 fitness = 0.7 * classification_error + 0.3 * feature_ratio
            return fitness
        except Exception as e:
            # print(f"⚠️ Evaluation failed: {e}")
            return 1.0

    def update_wolf_position(self, current_pos, alpha_pos, beta_pos, delta_pos, a, n_features):
        """Updates the position of a wolf based on the alpha, beta, and delta positions (GWO core)."""

        def calculate_new_position_component(leader_pos, current_pos, a, n_features):
            C = 2 * np.random.rand(n_features)
            A = 2 * a * np.random.rand(n_features) - a

            D = np.abs(C * leader_pos - current_pos)
            X1 = leader_pos - A * D
            return X1

        # Calculate steps towards Alpha, Beta, and Delta wolves
        X_alpha = calculate_new_position_component(alpha_pos, current_pos, a, n_features)
        X_beta = calculate_new_position_component(beta_pos, current_pos, a, n_features)
        X_delta = calculate_new_position_component(delta_pos, current_pos, a, n_features)

        # New position is the average of the steps
        new_pos = (X_alpha + X_beta + X_delta) / 3

        # Clamp positions to the search space [0, 1]
        new_pos = np.clip(new_pos, 0, 1)
        return new_pos

    def optimize(self, X_train, y_train, X_val, y_val, n_classes, n_features):
        print("🚀 Starting GWO-ANN-SSN Feature Selection")
        print("=" * 60)

        population = self.initialize_population(n_features)

        alpha_pos, beta_pos, delta_pos = None, None, None
        alpha_fitness, beta_fitness, delta_fitness = float('inf'), float('inf'), float('inf')
        best_accuracy = 0.0
        no_improvement_count = 0

        for generation in range(self.max_generations):
            print(f"\n🔄 Generation {generation + 1}/{self.max_generations}")
            fitness_values = []
            current_population_masks = []

            # 'a' decreases linearly from 2 to 0
            a = 2 - generation * (2 / self.max_generations)

            # Step 1: Evaluate current population
            for i, chromosome in enumerate(population):
                X_train_subset, selected_indices, binary_mask = self.select_feature_subset(chromosome, X_train)
                X_val_subset, _, _ = self.select_feature_subset(chromosome, X_val)
                feature_ratio = len(selected_indices) / n_features

                model, _ = self.classify_with_perceptron(X_train_subset, y_train, X_val_subset, y_val, n_classes, selected_indices)
                fitness = self.evaluate_with_ssn_norm(model, X_val_subset, y_val, feature_ratio)

                fitness_values.append(fitness)
                current_population_masks.append(binary_mask)

                if model is not None:
                    del model
                gc.collect()

            fitness_array = np.array(fitness_values)

            # Step 2: Identify Alpha, Beta, Delta (the top 3 best solutions)
            sorted_indices = np.argsort(fitness_array)

            # Update leaders and check for global improvement
            new_alpha_fitness = fitness_array[sorted_indices[0]]
            if new_alpha_fitness < alpha_fitness:
                alpha_fitness = new_alpha_fitness
                alpha_pos = population[sorted_indices[0]].copy()
                alpha_mask = current_population_masks[sorted_indices[0]]
                no_improvement_count = 0

                # Recalculate and print best accuracy
                try:
                    X_val_subset, selected_indices, _ = self.select_feature_subset(alpha_pos, X_val)
                    current_model, _ = self.classify_with_perceptron(X_train[:, alpha_mask==1], y_train, X_val_subset, y_val, n_classes, selected_indices)
                    if current_model:
                        preds = np.argmax(current_model.predict(X_val_subset, verbose=0), axis=1)
                        best_accuracy = accuracy_score(y_val.flatten(), preds)
                        del current_model
                        print(f"   ✅ New alpha! Fitness: {alpha_fitness:.4f}, Accuracy: {best_accuracy:.4f}, Features: {len(selected_indices)}")
                except:
                    print(f"   ✅ New alpha! Fitness: {alpha_fitness:.4f}, Features: {len(selected_indices)} (accuracy calc failed)")

            else:
                no_improvement_count += 1

            # Ensure at least three leaders exist for position update
            if len(sorted_indices) >= 3:
                beta_fitness = fitness_array[sorted_indices[1]]
                beta_pos = population[sorted_indices[1]].copy()
                delta_fitness = fitness_array[sorted_indices[2]]
                delta_pos = population[sorted_indices[2]].copy()
            else:
                # Fallback if population size is too small (should not happen with default parameters)
                beta_pos = alpha_pos.copy() if alpha_pos is not None else np.random.rand(n_features)
                delta_pos = alpha_pos.copy() if alpha_pos is not None else np.random.rand(n_features)

            # Check termination
            if (generation >= self.max_generations or no_improvement_count >= 30):
                break

            # Ensure leaders are not None before update
            if alpha_pos is None or beta_pos is None or delta_pos is None:
                # If leaders are not initialized (e.g., all models failed in Gen 1), re-initialize population
                population = self.initialize_population(n_features)
                continue

            # Step 3: Update positions of all other wolves
            new_population = []
            for i, current_pos in enumerate(population):
                # Elitism: keep the best three wolves
                if i in sorted_indices[:3]:
                    new_population.append(current_pos.copy())
                else:
                    new_pos = self.update_wolf_position(current_pos, alpha_pos, beta_pos, delta_pos, a, n_features)
                    new_population.append(new_pos)

            population = np.array(new_population)

            avg_fitness, current_best = np.mean(fitness_array), np.min(fitness_array)
            population_diversity = np.mean(np.std(population > 0.5, axis=0))
            print(f"   📊 Stats - Best: {current_best:.4f}, Avg: {avg_fitness:.4f}, Diversity: {population_diversity:.4f}")

        print("\n" + "=" * 60)
        print("🎯 GWO-ANN-SSN COMPLETED")
        print("=" * 60)

        if alpha_pos is None:
             # Fallback: select all features if GWO failed
             alpha_pos = np.ones(n_features)
             alpha_fitness = 1.0

        # Final best features are the binarized alpha position
        X_final_subset, final_selected_indices, final_best_mask = self.select_feature_subset(alpha_pos, X_train)

        print(f"Best Fitness: {alpha_fitness:.4f}")
        print(f"Best Accuracy: {best_accuracy:.4f}")
        print(f"Selected Features: {len(final_selected_indices)}/{n_features}")
        print(f"Final Generation: {generation + 1}")

        return final_best_mask, alpha_fitness, best_accuracy, final_selected_indices

print("✅ GWO-ANN-SSN Feature Selector class defined!")


# Cell 8: Main Pipeline Execution
# ===============================
print("🚀 Starting Unified GWO-ANN-SSN Pipeline")
print("=" * 60)

# Step 1: Load and extract features
X_train_feats, y_train, X_val_feats, y_val, X_test_feats, y_test, mapping = load_and_extract_features()
n_classes = len(mapping)

# Step 2: Feature standardization
print("🔧 Standardizing features...")
scaler = StandardScaler()
X_train_feats = scaler.fit_transform(X_train_feats)
X_val_feats = scaler.transform(X_val_feats)
X_test_feats = scaler.transform(X_test_feats)

# Step 3: Apply PCA
print("📊 Applying PCA...")
pca_subset_size = min(2000, len(X_train_feats))
pca_indices = np.random.choice(len(X_train_feats), pca_subset_size, replace=False)

pca_temp = PCA(n_components=PCA_VARIANCE, random_state=RANDOM_SEED)
pca_temp.fit(X_train_feats[pca_indices])
n_components_95 = pca_temp.n_components_

pca = PCA(n_components=n_components_95, random_state=RANDOM_SEED)
pca.fit(X_train_feats[pca_indices])

X_train_pca = pca.transform(X_train_feats)
X_val_pca = pca.transform(X_val_feats)
X_test_pca = pca.transform(X_test_feats)

n_features = X_train_pca.shape[1]
print(f"✅ PCA completed: {n_features} features (from {X_train_feats.shape[1]})")

# Save PCA and scaler
pca_file = os.path.join(CONFIG['features_dir'], "pca_scaler.pkl")
with open(pca_file, 'wb') as f:
    pickle.dump({'pca': pca, 'scaler': scaler}, f)
print("💾 Saved PCA and Scaler")

print("✅ Data preprocessing completed!")

# Cell 9: EFFICIENT GWO-ANN-SSN Feature Selection
# ==============================================
print("🚀 Starting EFFICIENT GWO-ANN-SSN Feature Selection...")
print("=" * 60)

# Adjusted parameters for a better trade-off between efficiency and performance
EFFICIENT_POPULATION_SIZE = 8
EFFICIENT_MAX_GENERATION = 10

EFFICIENT_EVAL_EPOCHS = 5 # Increased from 3 to 5 for better evaluation
EVAL_EPOCHS = EFFICIENT_EVAL_EPOCHS  # Override for efficiency

print(f"⚡ RUNNING IN EFFICIENT MODE ⚡")
print(f"   • Population: {EFFICIENT_POPULATION_SIZE}")
print(f"   • Generations: {EFFICIENT_MAX_GENERATION}")
print(f"   • Evaluation Epochs: {EVAL_EPOCHS}")
print("-" * 60)

# --- Instantiation changed to GWO selector ---
gwo_selector = GWO_ANN_SSN_FeatureSelector(
    population_size=EFFICIENT_POPULATION_SIZE,
    max_generations=EFFICIENT_MAX_GENERATION
)

try:
    best_mask, best_fitness, best_accuracy, selected_indices = gwo_selector.optimize(
        X_train_pca, y_train, X_val_pca, y_val, n_classes, n_features
    )
    print("✅ GWO-ANN-SSN feature selection completed!")

except UnboundLocalError as e:
    print(f"❌ Error in GWO optimization: {e}")
    print("🔄 Creating fallback solution (Variance Threshold)...")
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold()
    selector.fit(X_train_pca)
    variances = selector.variances_
    n_select = max(1, n_features // 2)
    selected_indices = np.argsort(variances)[-n_select:]
    best_mask = np.zeros(n_features)
    best_mask[selected_indices] = 1
    best_fitness, best_accuracy = 0.5, 0.5
    print(f"🔄 Using fallback: {len(selected_indices)} features selected")

except Exception as e:
    print(f"❌ Unexpected error in GWO: {e}")
    print("🔄 Creating random feature selection as fallback...")
    n_select = max(1, n_features // 2)
    selected_indices = np.random.choice(n_features, n_select, replace=False)
    best_mask = np.zeros(n_features)
    best_mask[selected_indices] = 1
    best_fitness, best_accuracy = 0.5, 0.5
    print(f"🔄 Using random selection: {len(selected_indices)} features selected")

print(f"🎯 Final selection: {len(selected_indices)} features out of {n_features}")

# Restore original value
EVAL_EPOCHS = 10
print(f"\nℹ️ Restored EVAL_EPOCHS to original value ({EVAL_EPOCHS})")

# Cell 10: Final Model Training
# =============================
print("\n🎯 Training Final Model with Selected Features...")
print("=" * 60)

# Prepare data with selected features
X_train_final = np.vstack([X_train_pca, X_val_pca])[:, selected_indices]
y_train_final = np.vstack([y_train, y_val])
X_test_final = X_test_pca[:, selected_indices]

print(f"📊 Final training set: {X_train_final.shape}")
print(f"📊 Test set: {X_test_final.shape}")
print(f"🔧 Selected features: {X_train_final.shape[1]}")

# Build and train final model
final_model = build_ssn_perceptron(X_train_final.shape[1], n_classes, hidden_units=512, dropout_rate=0.5)

print(f"🔧 Training final model on combined train+val data for {FINAL_EPOCHS} epochs...")
history = final_model.fit(
    X_train_final, y_train_final,
    epochs=FINAL_EPOCHS, # FINAL_EPOCHS is now 100
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_split=0.0  # No validation during final training
)

print("✅ Final model training completed!")

# Cell 11: Evaluation and Results Summary
# =======================================
print("\n📈 Evaluating on completely held-out TEST set...")
print("=" * 60)

# Validation performance for reference
X_val_selected = X_val_pca[:, selected_indices]
val_preds = np.argmax(final_model.predict(X_val_selected, verbose=0), axis=1)
val_accuracy = accuracy_score(y_val.flatten(), val_preds)

# Test performance (true performance)
test_preds = np.argmax(final_model.predict(X_test_final, verbose=0), axis=1)
test_accuracy = accuracy_score(y_test.flatten(), test_preds)

print(f"🏆 VALIDATION Accuracy (for reference): {val_accuracy:.4f}")
print(f"🎯 TEST Accuracy (true performance): {test_accuracy:.4f}")

# Final results summary
print("\n" + "=" * 60)
print(f"🎯 FINAL GWO-ANN-SSN RESULTS - {DATASET_CHOICE.upper()}")
print("=" * 60)

print(f"📊 Dataset Usage:")
print(f"   • Train: {X_train_pca.shape[0]} samples (feature selection)")
print(f"   • Validation: {X_val_pca.shape[0]} samples (fitness evaluation)")
print(f"   • Test: {X_test_pca.shape[0]} samples (FINAL evaluation)")
print(f"   • Final Training: {X_train_final.shape[0]} samples (train+val)")

print(f"\n🎯 Performance Metrics:")
print(f"   • Best Fitness: {best_fitness:.4f}")
print(f"   • Validation Accuracy: {val_accuracy:.4f}")
print(f"   • 🏆 TEST Accuracy: {test_accuracy:.4f} ← TRUE PERFORMANCE")

print(f"\n🔧 Feature Selection:")
print(f"   • Selected Features: {len(selected_indices)}/{n_features}")
print(f"   • Feature Reduction: {((n_features - len(selected_indices)) / n_features * 100):.1f}%")

# Save results
results = {
    'dataset': DATASET_CHOICE,
    'test_accuracy': test_accuracy,
    'validation_accuracy': val_accuracy,
    'gwo_best_fitness': best_fitness,
    'selected_features_count': len(selected_indices),
    'total_features': n_features,
    'selected_indices': selected_indices,
    'dataset_sizes': {
        'train': X_train_pca.shape[0],
        'validation': X_val_pca.shape[0],
        'test': X_test_pca.shape[0],
        'final_training': X_train_final.shape[0]
    },
    'classes': mapping
}

results_file = os.path.join(CONFIG['features_dir'], "gwo_ann_ssn_results.pkl")
with open(results_file, 'wb') as f:
    pickle.dump(results, f)

print(f"\n💾 Results saved to: {results_file}")
print("✅ Unified pipeline completed successfully!")

# Cell 12: Results Consolidation
# ==============================
print("\n" + "=" * 60)
print("📊 RESULTS CONSOLIDATION")
print("=" * 60)

def save_consolidated_results(test_accuracy, val_accuracy, best_fitness, selected_indices,
                              n_features, n_classes, mapping, X_train_pca, X_test_pca):
    """Save all results to consolidated files"""

    # Create results entry for current dataset
    results_entry = {
        'dataset': DATASET_CHOICE,
        'performance_metrics': {
            'test_accuracy': float(test_accuracy),
            'validation_accuracy': float(val_accuracy),
            'best_fitness': float(best_fitness),
            'performance_gap': float(test_accuracy - val_accuracy)
        },
        'feature_selection': {
            'selected_features_count': len(selected_indices),
            'total_features': n_features,
            'feature_reduction_percentage': float((n_features - len(selected_indices)) / n_features * 100),
            'selected_features_ratio': float(len(selected_indices) / n_features)
        },
        'dataset_info': {
            'num_classes': n_classes,
            'class_names': list(mapping.values()),
            'training_samples': int(X_train_pca.shape[0]),
            'test_samples': int(X_test_pca.shape[0])
        },
        'timestamp': str(pd.Timestamp.now())
    }

    # 1. Append to main JSON results file
    main_json_file = "gwo_ann_ssn_datasets_results.json"

    # Load existing results or create new
    if os.path.exists(main_json_file):
        with open(main_json_file, 'r') as f:
            all_results = json.load(f)
    else:
        all_results = {}

    # Add/update current dataset results
    all_results[DATASET_CHOICE] = results_entry

    # Save updated results
    with open(main_json_file, 'w') as f:
        json.dump(all_results, f, indent=4)

    print(f"💾 Results appended to: {main_json_file}")

    # 2. Update summary text file
    summary_file = "gwo_ann_ssn_datasets_summary.txt"

    with open(summary_file, 'w') as f:
        f.write("=" * 70 + "\n")
        f.write("GWO-ANN-SSN ALL DATASETS SUMMARY\n")
        f.write("=" * 70 + "\n\n")

        # Write header
        f.write(f"{'Dataset':<12} {'Test Acc':<10} {'Val Acc':<10} {'Features':<12} {'Reduction':<12} {'Classes':<8}\n")
        f.write("-" * 70 + "\n")

        # Write all dataset entries
        for dataset_name, dataset_results in all_results.items():
            perf = dataset_results['performance_metrics']
            feat = dataset_results['feature_selection']
            info = dataset_results['dataset_info']

            f.write(f"{dataset_name:<12} {perf['test_accuracy']:<10.4f} {perf['validation_accuracy']:<10.4f} "
                    f"{feat['selected_features_count']}/{feat['total_features']:<12} "
                    f"{feat['feature_reduction_percentage']:<12.1f}% "
                    f"{info['num_classes']:<8}\n")

        f.write("\n" + "=" * 70 + "\n")
        f.write("LATEST RUN DETAILS:\n")
        f.write("=" * 70 + "\n\n")

        # Add details for current run
        f.write(f"Dataset: {DATASET_CHOICE}\n")
        f.write(f"Test Accuracy: {test_accuracy:.4f}\n")
        f.write(f"Validation Accuracy: {val_accuracy:.4f}\n")
        f.write(f"Selected Features: {len(selected_indices)}/{n_features}\n")
        f.write(f"Feature Reduction: {((n_features - len(selected_indices)) / n_features * 100):.1f}%\n")
        f.write(f"Number of Classes: {n_classes}\n")
        f.write(f"Best Fitness: {best_fitness:.4f}\n")
        f.write(f"Classes: {', '.join(list(mapping.values()))}\n")
        f.write(f"Timestamp: {results_entry['timestamp']}\n")

    print(f"📋 Summary updated in: {summary_file}")

    # Print current results to console
    print(f"\n📊 {DATASET_CHOICE.upper()} RESULTS:")
    print("-" * 50)
    print(f"Test Accuracy:       {test_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Selected Features:   {len(selected_indices)}/{n_features}")
    print(f"Feature Reduction:   {((n_features - len(selected_indices)) / n_features * 100):.1f}%")
    print(f"Best Fitness:        {best_fitness:.4f}")

    return results_entry

# Execute results consolidation
try:
    consolidated_results = save_consolidated_results(
        test_accuracy, val_accuracy, best_fitness, selected_indices,
        n_features, n_classes, mapping, X_train_pca, X_test_pca
    )
    print("\n✅ All results consolidated successfully!")

except Exception as e:
    print(f"⚠️ Error in consolidating results: {e}")
    print("Continuing with existing results...")

print("=" * 60)

✅ All imports successful!
Available datasets:
1. plants
2. aid
3. lc25000
Select dataset (enter number or name): 3
Selected dataset: lc25000
Are you running this code in Google Colab or locally? (colab/local): colab
✅ Configuration set for LC25000 dataset!
Dataset URL: https://www.kaggle.com/datasets/javaidahmadwani/lc25000


100%|██████████| 1.76G/1.76G [00:51<00:00, 36.5MB/s]



✅ LC25000 dataset downloaded successfully!
✅ Data loading functions defined!
✅ Feature extraction utilities defined!
✅ SSN implementation complete!
✅ GWO-ANN-SSN Feature Selector class defined!
🚀 Starting Unified GWO-ANN-SSN Pipeline
🚀 Starting feature extraction pipeline...
📂 Loading LC25000 dataset...
✅ Loaded 2499 images from ./lc25000-datasets/lc25000/lung_colon_image_set/Test Set
✅ Loaded 12500 images from ./lc25000-datasets/lc25000/lung_colon_image_set/Train and Validation Set
🎯 Classes found: 5
📊 Dataset sizes - Train: 10000, Val: 2500, Test: 2499
🔧 Extracting features for train...
    Processed 10/79 batches
    Processed 20/79 batches
    Processed 30/79 batches
    Processed 40/79 batches
    Processed 50/79 batches
    Processed 60/79 batches
    Processed 70/79 batches
    Processed 79/79 batches
💾 Saved features for train: (10000, 512)
🔧 Extracting features for val...
    Processed 10/20 batches
    Processed 20/20 batches
💾 Saved features for val: (2500, 512)
🔧 Extractin

In [3]:
import json
import os

# --- Configuration (Ensure these match your saved filenames) ---
SUMMARY_FILE = "gwo_ann_ssn_datasets_summary.txt"
JSON_FILE = "gwo_ann_ssn_datasets_results.json"
# ----------------------------------------------------------------

print("="*60)
print("📄 CONSOLIDATED RESULTS SUMMARY (TXT)")
print("="*60)

if os.path.exists(SUMMARY_FILE):
    with open(SUMMARY_FILE, 'r') as f:
        summary_content = f.read()
    print(summary_content)
else:
    print(f"❌ Error: Summary file '{SUMMARY_FILE}' not found.")
    print("Please ensure the file was created in the correct directory.")

print("\n" + "="*60)
print("💡 FULL JSON RESULTS (DETAIL)")
print("="*60)

if os.path.exists(JSON_FILE):
    with open(JSON_FILE, 'r') as f:
        json_results = json.load(f)

    # Use json.dumps for neat, formatted printing
    print(json.dumps(json_results, indent=4))
else:
    print(f"❌ Error: JSON file '{JSON_FILE}' not found.")
    print("Please ensure the file was created in the correct directory.")

print("="*60)

📄 CONSOLIDATED RESULTS SUMMARY (TXT)
GWO-ANN-SSN ALL DATASETS SUMMARY

Dataset      Test Acc   Val Acc    Features     Reduction    Classes 
----------------------------------------------------------------------
plants       0.9260     0.9809     142/331          57.1        % 30      
aid          0.7300     0.9980     132/327          59.6        % 30      
lc25000      0.9768     0.9968     124/325          61.8        % 5       

LATEST RUN DETAILS:

Dataset: lc25000
Test Accuracy: 0.9768
Validation Accuracy: 0.9968
Selected Features: 124/325
Feature Reduction: 61.8%
Number of Classes: 5
Best Fitness: 0.1363
Classes: colon_aca, colon_n, lung_aca, lung_n, lung_scc
Timestamp: 2025-10-31 12:27:25.334730


💡 FULL JSON RESULTS (DETAIL)
{
    "plants": {
        "dataset": "plants",
        "performance_metrics": {
            "test_accuracy": 0.9259506337558372,
            "validation_accuracy": 0.9808580858085808,
            "best_fitness": 0.24883291954573095,
            "performan